<a href="https://colab.research.google.com/github/ReneeDu320/deep_learning/blob/main/pytorch_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch 中的循环神经网络模块
前面我们讲了循环神经网络的基础知识和网络结构，下面我们教大家如何在 pytorch 下构建循环神经网络，因为 pytorch 的动态图机制，使得循环神经网络非常方便。

## 一般的 RNN

![](https://ws1.sinaimg.cn/large/006tKfTcly1fmt9xz889xj30kb07nglo.jpg)

对于最简单的 RNN，我们可以使用下面两种方式去调用，分别是 `torch.nn.RNNCell()` 和 `torch.nn.RNN()`，这两种方式的区别在于 `RNNCell()` 只能接受序列中单步的输入，且必须传入隐藏状态，而 `RNN()` 可以接受一个序列的输入，默认会传入全 0 的隐藏状态，也可以自己申明隐藏状态传入。

`RNN()` 里面的参数有

- input_size 表示输入 $x_t$ 的特征维度

- hidden_size 表示输出的特征维度

- num_layers 表示网络的层数

- nonlinearity 表示选用的非线性激活函数，默认是 'tanh'

- bias 表示是否使用偏置，默认使用

- batch_first 表示输入数据的形式，默认是 False，就是这样形式，(seq, batch, feature)，也就是将序列长度放在第一位，batch 放在第二位

- dropout 表示是否在输出层应用 dropout

- bidirectional 表示是否使用双向的 rnn，默认是 False

对于 `RNNCell()`，里面的参数就少很多，只有 input_size，hidden_size，bias 以及 nonlinearity

In [1]:
import torch
from torch.autograd import Variable
from torch import nn

In [2]:
# 定义一个单步的 rnn
rnn_single = nn.RNNCell(input_size=100, hidden_size=200)

In [3]:
# 访问其中的参数
rnn_single.weight_hh

Parameter containing:
tensor([[-0.0033, -0.0152,  0.0307,  ..., -0.0344,  0.0261,  0.0541],
        [ 0.0309, -0.0070,  0.0639,  ...,  0.0002,  0.0221,  0.0698],
        [-0.0592, -0.0450,  0.0249,  ..., -0.0252, -0.0147, -0.0516],
        ...,
        [ 0.0613,  0.0462,  0.0278,  ..., -0.0163,  0.0490,  0.0362],
        [ 0.0679, -0.0689,  0.0192,  ...,  0.0059, -0.0541, -0.0616],
        [ 0.0692, -0.0293,  0.0652,  ..., -0.0644, -0.0603, -0.0150]],
       requires_grad=True)

In [4]:
# 构造一个序列，长为 6，batch 是 5， 特征是 100
x = Variable(torch.randn(6, 5, 100)) # 这是 rnn 的输入格式

In [5]:
# 定义初始的记忆状态
h_t = Variable(torch.zeros(5, 200))

In [6]:
# 传入 rnn
out = []
for i in range(6): # 通过循环 6 次作用在整个序列上
    h_t = rnn_single(x[i], h_t)
    out.append(h_t)

In [12]:
h_t.shape

torch.Size([5, 200])

In [9]:
len(out)

6

In [10]:
out[0].shape # 每个输出的维度

torch.Size([5, 200])

可以看到经过了 rnn 之后，隐藏状态的值已经被改变了，因为网络记忆了序列中的信息，同时输出 6 个结果

下面我们看看直接使用 `RNN` 的情况

In [13]:
rnn_seq = nn.RNN(100, 200)

In [14]:
# 访问其中的参数
rnn_seq.weight_hh_l0

Parameter containing:
tensor([[ 4.7964e-02, -1.9068e-02, -2.4784e-02,  ..., -5.4594e-02,
          2.1527e-02,  1.9974e-02],
        [-3.4415e-02, -7.0402e-03, -2.8575e-02,  ...,  1.6532e-02,
          4.5620e-02,  3.6163e-02],
        [ 6.2873e-02, -2.3012e-02, -3.8514e-02,  ..., -6.8905e-02,
          2.2811e-02,  2.1178e-02],
        ...,
        [-3.4806e-03, -5.2392e-02,  5.1187e-02,  ...,  4.6457e-02,
         -5.2636e-03, -4.5401e-05],
        [-6.6950e-04,  4.5032e-03, -2.3971e-02,  ..., -4.4612e-02,
         -1.3389e-02,  3.3729e-02],
        [-8.5752e-03, -1.4127e-02,  4.8226e-02,  ..., -2.8765e-02,
         -1.0604e-02,  1.2273e-02]], requires_grad=True)

In [15]:
out, h_t = rnn_seq(x) # 使用默认的全 0 隐藏状态

In [18]:
h_t.shape

torch.Size([1, 5, 200])

In [19]:
len(out)

6

这里的 h_t 是网络最后的隐藏状态，网络也输出了 6 个结果

In [20]:
# 自己定义初始的隐藏状态
h_0 = Variable(torch.randn(1, 5, 200))

这里的隐藏状态的大小有三个维度，分别是 (num_layers * num_direction, batch, hidden_size)

In [21]:
out, h_t = rnn_seq(x, h_0)

In [23]:
h_t.shape

torch.Size([1, 5, 200])

In [24]:
out.shape

torch.Size([6, 5, 200])

同时输出的结果也是 (seq, batch, feature)

一般情况下我们都是用 `nn.RNN()` 而不是 `nn.RNNCell()`，因为 `nn.RNN()` 能够避免我们手动写循环，非常方便，同时如果不特别说明，我们也会选择使用默认的全 0 初始化隐藏状态

## LSTM

![](https://ws1.sinaimg.cn/large/006tKfTcly1fmt9qj3uhmj30iz07ct90.jpg)

LSTM 和基本的 RNN 是一样的，他的参数也是相同的，同时他也有 `nn.LSTMCell()` 和 `nn.LSTM()` 两种形式，跟前面讲的都是相同的，我们就不再赘述了，下面直接举个小例子

In [25]:
lstm_seq = nn.LSTM(50, 100, num_layers=2) # 输入维度 50，输出 100，两层

In [ ]:
lstm_seq.weight_hh_l0 # 第一层的 h_t 权重

Parameter containing:
1.00000e-02 *
 3.8420  5.7387  6.1351  ...   1.2680  0.9890  1.3037
-4.2301  6.8294 -4.8627  ...  -6.4147  4.3015  8.4103
 9.4411  5.0195  9.8620  ...  -1.6096  9.2516 -0.6941
          ...             ⋱             ...          
 1.2930 -1.3300 -0.9311  ...  -6.0891 -0.7164  3.9578
 9.0435  2.4674  9.4107  ...  -3.3822 -3.9773 -3.0685
-4.2039 -8.2992 -3.3605  ...   2.2875  8.2163 -9.3277
[torch.FloatTensor of size 400x100]

LSTM in Tensorflow: https://jasdeep06.github.io/posts/Understanding-LSTM-in-Tensorflow-MNIST/

**小练习：想想为什么这个系数的大小是 (400, 100)**

weight_hh_l[k] – the learnable hidden-hidden weights of the k_th
  layer (W_hi|W_hf|W_hg|W_ho), of shape (4*hidden_size, hidden_size). If proj_size > 0 was specified, the shape will be (4**hidden_size, proj_size).

In [26]:
lstm_input = Variable(torch.randn(10, 3, 50)) # 序列 10，batch 是 3，输入维度 50

In [27]:
out, (h, c) = lstm_seq(lstm_input) # 使用默认的全 0 隐藏状态

注意这里 LSTM 输出的隐藏状态有两个，h 和 c，就是上图中的每个 cell 之间的两个箭头，这两个隐藏状态的大小都是相同的，(num_layers * direction, batch, feature)

In [28]:
h.shape # 两层，Batch 是 3，特征是 100

torch.Size([2, 3, 100])

In [29]:
c.shape

torch.Size([2, 3, 100])

In [30]:
out.shape

torch.Size([10, 3, 100])

我们可以不使用默认的隐藏状态，这是需要传入两个张量

In [31]:
h_init = Variable(torch.randn(2, 3, 100))
c_init = Variable(torch.randn(2, 3, 100))

In [32]:
out, (h, c) = lstm_seq(lstm_input, (h_init, c_init))

In [33]:
h.shape

torch.Size([2, 3, 100])

In [34]:
c.shape

torch.Size([2, 3, 100])

In [35]:
out.shape

torch.Size([10, 3, 100])

# GRU
![](https://ws3.sinaimg.cn/large/006tKfTcly1fmtaj38y9sj30io06bmxc.jpg)

GRU 和前面讲的这两个是同样的道理，就不再细说，还是演示一下例子

In [ ]:
gru_seq = nn.GRU(10, 20)
gru_input = Variable(torch.randn(3, 32, 10))

out, h = gru_seq(gru_input)

In [ ]:
gru_seq.weight_hh_l0

Parameter containing:
 0.0766 -0.0548 -0.2008  ...  -0.0250 -0.1819  0.1453
-0.1676  0.1622  0.0417  ...   0.1905 -0.0071 -0.1038
 0.0444 -0.1516  0.2194  ...  -0.0009  0.0771  0.0476
          ...             ⋱             ...          
 0.1698 -0.1707  0.0340  ...  -0.1315  0.1278  0.0946
 0.1936  0.1369 -0.0694  ...  -0.0667  0.0429  0.1322
 0.0870 -0.1884  0.1732  ...  -0.1423 -0.1723  0.2147
[torch.FloatTensor of size 60x20]

In [ ]:
h.shape

torch.Size([1, 32, 20])

In [ ]:
out.shape

torch.Size([3, 32, 20])

## example code

In [ ]:
import torch.nn as nn
from torch.autograd import Variable


class MyModel(nn.Module):
	def __init__(self, ...):
	...
	self.lstm = nn.LSTM(embedding_length, hidden_size)
	self.label = nn.Linear(hidden_size, output_size)


	def forward(self):


	h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
	c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())


	output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))


	return self.label(final_hidden_state[-1]) 

wandb.watch(model)


def train_model(model, train_iter, epoch):
    ...
    model.train()
    for idx, batch in enumerate(train_iter):
	...
        prediction = model(text)
        loss = loss_fn(prediction, target)
        wandb.log({"Training Loss": loss.item()})
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        wandb.log({"Training Accuracy": acc.item()})
        ...


https://cnvrg.io/pytorch-lstm/

https://machinelearningmastery.com/lstm-for-time-series-prediction-in-pytorch/


